<a href="https://colab.research.google.com/github/aymuos/masters-practise-repo/blob/main/TERM3/AI_at_Scale/Assignment1/question1/q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

q1

In [6]:
! pip install -q pyspark

In [7]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType


In [8]:

spark = SparkSession.builder \
    .appName("assignment1_CH24M571") \
    .master("local[*]") \
    .config("spark.executor.memory", "512m") \
    .config("spark.executor.core", "4") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.instances", "2")\
    .config("spark.driver.core", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")


# # spark = SparkSession.builder \: This starts the process of building a SparkSession.
# .appName("Assignment1") \: This sets the name of the Spark application to "Assignment1".
# .master("local[*]") \: This configures Spark to run locally on your machine using all available cores.
# .config("spark.executor.memory", "512m") \: This sets the amount of memory allocated to each executor (the processes that perform the actual computations) to 512 megabytes.
# .config("spark.executor.core", "4") \: This sets the number of cores used by each executor to 4.
# .config("spark.driver.memory", "2g") \: This sets the amount of memory allocated to the Spark driver (the process that coordinates the execution) to 2 gigabytes.
# .config("spark.executor.instances", "2")\: This sets the number of executor instances to 2.
# .config("spark.driver.core", "2") \: This sets the number of cores used by the driver to 2.
# .getOrCreate(): This retrieves an existing SparkSession if one is already running, or creates a new one if not.
# spark.sparkContext.setLogLevel("INFO"): This sets the logging level for the Spark context to "INFO", which means informational messages will be displayed.

In [ ]:
# setting the schemas for the 3 csv s

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType, StructType

# Customers schema
customer_schema = StructType([
    StructField("customer_id", StringType(), False),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("email", StringType(), True),
    StructField("preferences", StringType(), True),  # Will parse to ArrayType later
    StructField("address", StringType(), True)       # Will parse to StructType later
])

# Orders schema
order_schema = StructType([
    StructField("order_id", StringType(), False),
    StructField("customer_id", StringType(), True),
    StructField("order_date", StringType(), True),
    StructField("items", StringType(), True)
    StructField("shipping_address", StringType(), True),
    StructField("total_amount", FloatType(), True)
])

# Products schema
product_schema = StructType([
    StructField("product_id", StringType(), False),
    StructField("name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", FloatType(), True),
    StructField("tags", IntegerType(), True)
])

In [ ]:
# defining file paths

customer_csv= './customers.csv'
orders_csv = './orders.csv'
product_csv = './product.csv'


# read csv file
customer_df = spark.read.csv(customer_csv, header=True, inferSchema=True)
# show the dataframe
customer_df.show()

+-----------+-----------+---+--------------------+---------------+--------------------+
|customer_id|       name|age|               email|    preferences|             address|
+-----------+-----------+---+--------------------+---------------+--------------------+
| CUST000001| Customer_1| 61|customer1@example...|   "[""sports""|          ""travel""|
| CUST000002| Customer_2| 55|customer2@example...|           NULL|"{""street"": ""5...|
| CUST000003| Customer_3| 79|customer3@example...|   "[""movies""|           ""books""|
| CUST000004| Customer_4| 73|customer4@example...|    "[""books""|         ""fitness""|
| CUST000005| Customer_5| 80|customer5@example...|    "[""music""|           ""books""|
| CUST000006| Customer_6| 44|customer6@example...|   "[""travel""|       ""fitness""]"|
| CUST000007| Customer_7| 57|customer7@example...|    "[""music""|          ""movies""|
| CUST000008| Customer_8| 20|                NULL|    "[""books""|         ""fitness""|
| CUST000009| Customer_9| 57|cus

In [ ]:
# read csv file
orders_df = spark.read.csv(orders_csv, header=True, inferSchema=True)
# show the dataframe
orders_df.show()

+----------+-----------+----------+--------------------+----------------+--------------------+
|  order_id|customer_id|order_date|               items|shipping_address|        total_amount|
+----------+-----------+----------+--------------------+----------------+--------------------+
|ORD0000001| CUST000046|2023-08-16|"[{""product_id""...| ""quantity"": 3| ""price"": 370.9...|
|ORD0000002| CUST000027|2023-11-20|"[{""product_id""...| ""quantity"": 2|   ""price"": 40.45}|
|ORD0000003| CUST000093|2023-11-27|"[{""product_id""...| ""quantity"": 3|  ""price"": 428.33}|
|ORD0000004| CUST000017|2023-04-08|"[{""product_id""...| ""quantity"": 2|   ""price"": 176.2}|
|ORD0000005| CUST000092|2023-09-24|"[{""product_id""...| ""quantity"": 3|  ""price"": 370.91}|
|ORD0000006| CUST000029|2023-09-04|"[{""product_id""...| ""quantity"": 2| ""price"": 400.4...|
|ORD0000007| CUST000093|2023-10-22|"[{""product_id""...| ""quantity"": 1|  ""price"": 370.91}|
|ORD0000008| CUST000094|2023-10-20|"[{""product_id

In [ ]:
# read csv file
customer_df = spark.read.csv(customer_csv, header=True, inferSchema=True)
# show the dataframe
customer_df.show()

+-----------+-----------+---+--------------------+---------------+--------------------+
|customer_id|       name|age|               email|    preferences|             address|
+-----------+-----------+---+--------------------+---------------+--------------------+
| CUST000001| Customer_1| 61|customer1@example...|   "[""sports""|          ""travel""|
| CUST000002| Customer_2| 55|customer2@example...|           NULL|"{""street"": ""5...|
| CUST000003| Customer_3| 79|customer3@example...|   "[""movies""|           ""books""|
| CUST000004| Customer_4| 73|customer4@example...|    "[""books""|         ""fitness""|
| CUST000005| Customer_5| 80|customer5@example...|    "[""music""|           ""books""|
| CUST000006| Customer_6| 44|customer6@example...|   "[""travel""|       ""fitness""]"|
| CUST000007| Customer_7| 57|customer7@example...|    "[""music""|          ""movies""|
| CUST000008| Customer_8| 20|                NULL|    "[""books""|         ""fitness""|
| CUST000009| Customer_9| 57|cus